In [2]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import datetime
import datetime as dt
import pickle

In [2]:
#filter datasource and variables from param file
param=pd.read_excel('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/paramfile_ver1.0.xlsx',usecols=['columns', 'datatype', 'is_use', 'isdate', 'dateformat', 'datasource'])
param = param.drop(param[param['is_use'] == 'N'].index)

ls = ['aarp','ds', 'tu']
param = param[param['datasource'].isin(ls)]

print(param.shape)

#create empty dataframe with intented columns and their datatype as per param file
dic = dict(zip(param['columns'],param['datatype']))

new = pd.DataFrame({c: pd.Series(dtype=t) for c, t in dic.items()})
new.shape

#Addd the target columns
col_lst = list(new.columns)
col_lst.append('RESPONSE_CNT')
col_lst.append('RESPONSE_DATE')
col_lst.append('CONVERSION_CNT')
col_lst.append('CONVERSION_DATE')
col_lst.append('MONTH_YR')
col_lst.append('PSEUDO_INDIVIDUAL_ID')
col_lst.append('PSEUDO_HOUSEHOLD_ID')
col_lst.append('PSEUDO_RESIDENCE_ID')
col_lst.append('SEQ_NO')
col_lst.append('DROP_DATE')
col_lst.append('RANDOM_CELL_CODE')
col_lst.append('RANDOM_EXTRA')
col_lst.append('RANDOM_EXTRA_ALL_WEEKS')
col_lst.append('state')
col_lst.append('zip_code')

col_lst.append('AARP_PRIMARY_AGE')
col_lst.append('AARP_SECONDARY_AGE')
col_lst.append('AARP_PRIMARY_GENDER')
col_lst.append('AARP_SECONDARY_GENDER')
col_lst.append('AARP_PRIMARY_BIRTH_DT')
# col_lst.append('AARP_SECONDARY_BIRTH_DT')

len(col_lst)

(865, 6)


885

In [3]:
#Merge aarp datasets into one
aarp_new = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/gm_non_gm_append_aarp_mbr_variables.txt', sep='|')
aarp_old = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/gm_non_gm_append_aarp.txt', sep='|')

aarp=pd.merge(aarp_old.drop(columns=['AARP_AGE', 'AARP_SEC_AGE', 'AARP_IBX_NM_GENDER_INDV1_PR',
                                     'AARP_IBX_NM_GENDER_INDV2_PR','AARP_BIRTH_DT', 'AARP_SEC_BIRTH_DT',
                                     'AARP_ADDR_MOVE_DT', 'AARP_MARITAL_STATUS']),
              aarp_new[['AARP_PRIMARY_AGE', 'AARP_SECONDARY_AGE', 'AARP_PRIMARY_GENDER',
                        'AARP_SECONDARY_GENDER', 'AARP_PRIMARY_BIRTH_DT', 'AARP_SECONDARY_BIRTH_DT',
                        'AARP_ADDR_MOVE_DT', 'AARP_MARITAL_STATUS', 'DROP_DATE', 'PSEUDO_RESIDENCE_ID']],
              how='inner', on=['DROP_DATE', 'PSEUDO_RESIDENCE_ID'])

In [5]:
#source data from the file Merkle provided

tu = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/gm_non_gm_append_tu.csv')
ds = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/gm_non_gm_append_ds_v1.csv')

aarp1=aarp.drop(columns=['PSEUDO_INDIVIDUAL_ID', 'PSEUDO_HOUSEHOLD_ID', 'PSEUDO_RESIDENCE_ID'])
tu1 = tu.drop(columns=['PSEUDO_INDIVIDUAL_ID', 'PSEUDO_HOUSEHOLD_ID', 'PSEUDO_RESIDENCE_ID', 'state', 'zip_code']) 
ds1=ds.drop(columns=['PSEUDO_INDIVIDUAL_ID', 'PSEUDO_HOUSEHOLD_ID', 'PSEUDO_RESIDENCE_ID'])

aarp_ds_tu=pd.merge(pd.merge(aarp1, ds1, on='SEQ_NO', how='inner'), tu1, on='SEQ_NO', how='inner')

gm_append= pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/GM_model_all_vars_appended.csv')
gm_append.shape
gm_append.columns=map(str.upper, gm_append.columns)

aarp_ds_tu1=pd.merge(aarp_ds_tu, gm_append[['PSEUDO_INDIVIDUAL_ID', 'PSEUDO_HOUSEHOLD_ID', 
                    'PSEUDO_RESIDENCE_ID', 'MONTH_YR', 'RESPONSE_CNT', 'CONVERSION_CNT', 'RESPONSE_DATE', 
                     'CONVERSION_DATE', 'SEQ_NO','RANDOM_CELL_CODE','RANDOM_EXTRA', 'RANDOM_EXTRA_ALL_WEEKS']], on='SEQ_NO', how='inner')

aarp_ds_tu2=aarp_ds_tu1.drop_duplicates(subset = ['DROP_DATE', 'PSEUDO_RESIDENCE_ID'], keep = 'last').reset_index(drop = True)
print(aarp_ds_tu2.shape)

df1=aarp_ds_tu2[col_lst]
df1 = df1.loc[:,~df1.columns.duplicated()].copy()
print(df1.shape)

(852236, 1270)
(852236, 880)


In [7]:
df2 = new.append(df1).reset_index()
df2.drop(columns='index',inplace=True)
print(df2.shape)

(852236, 880)


In [21]:
df2['AARP_MEM_ORIGIN_DT'].head(10)

0    2020-05-13
1    2018-01-16
2    2017-01-23
3    2016-02-03
4    1986-11-12
5    2007-09-14
6    2011-11-03
7    2019-02-18
8    2020-02-15
9    2008-12-04
Name: AARP_MEM_ORIGIN_DT, dtype: object

In [8]:
#Handling variables which have incorrect formats, modifying to make them readable

df2['AARP_IBX_RET_PURCH_ACT_DT_CATEGORY1']=df2['AARP_IBX_RET_PURCH_ACT_DT_CAT1'].str[-2:]
df2['AARP_IBX_RET_PURCH_ACT_DT_CATEGORY2']=df2['AARP_IBX_RET_PURCH_ACT_DT_CAT2'].str[-2:]
df2['AARP_IBX_RET_PURCH_ACT_DT_CATEGORY3']=df2['AARP_IBX_RET_PURCH_ACT_DT_CAT3'].str[-2:]
df2['AARP_IBX_RET_PURCH_ACT_DT_CATEGORY4']=df2['AARP_IBX_RET_PURCH_ACT_DT_CAT4'].str[-2:]
df2['AARP_IBX_RET_PURCH_ACT_DT_CATEGORY5']=df2['AARP_IBX_RET_PURCH_ACT_DT_CAT5'].str[-2:]


df2['AARP_IBX_RET_PURCH_ACT_DT_CAT1']=df2['AARP_IBX_RET_PURCH_ACT_DT_CAT1'].str[:-2]
df2['AARP_IBX_RET_PURCH_ACT_DT_CAT2']=df2['AARP_IBX_RET_PURCH_ACT_DT_CAT2'].str[:-2]
df2['AARP_IBX_RET_PURCH_ACT_DT_CAT3']=df2['AARP_IBX_RET_PURCH_ACT_DT_CAT3'].str[:-2]
df2['AARP_IBX_RET_PURCH_ACT_DT_CAT4']=df2['AARP_IBX_RET_PURCH_ACT_DT_CAT4'].str[:-2]
df2['AARP_IBX_RET_PURCH_ACT_DT_CAT5']=df2['AARP_IBX_RET_PURCH_ACT_DT_CAT5'].str[:-2]

df2['AARP_IBX_RET_PURCH_ACT_DT_CAT1'] = df2['AARP_IBX_RET_PURCH_ACT_DT_CAT1'].str[:7] + '1'
df2['AARP_IBX_RET_PURCH_ACT_DT_CAT2'] = df2['AARP_IBX_RET_PURCH_ACT_DT_CAT2'].str[:7] + '1'
df2['AARP_IBX_RET_PURCH_ACT_DT_CAT3'] = df2['AARP_IBX_RET_PURCH_ACT_DT_CAT3'].str[:7] + '1'
df2['AARP_IBX_RET_PURCH_ACT_DT_CAT4'] = df2['AARP_IBX_RET_PURCH_ACT_DT_CAT4'].str[:7] + '1'
df2['AARP_IBX_RET_PURCH_ACT_DT_CAT5'] = df2['AARP_IBX_RET_PURCH_ACT_DT_CAT5'].str[:7] + '1'

df2['AARP_IBX_RET_PURCH_ACT_DT_CAT5'].values[df2['AARP_IBX_RET_PURCH_ACT_DT_CAT5'] == '20120001'] = np.nan
df2['AARP_IBX_RET_PURCH_ACT_DT_CAT5'].values[df2['AARP_IBX_RET_PURCH_ACT_DT_CAT5'] == '20010001'] = np.nan


df2['AARP_PAID_THROUGH_DT'].values[df2['AARP_PAID_THROUGH_DT'] == '2800-12-01'] = np.nan


df2['AARP_IBX_BASE_RECORD_VERIFY_DT'] = df2['AARP_IBX_BASE_RECORD_VERIFY_DT'].astype('str')
df2['AARP_IBX_HM_LOAN_DT_1']          = df2['AARP_IBX_HM_LOAN_DT_1'].astype('str')
df2['AARP_IBX_HM_PURCHASE_DT_PR']     = df2['AARP_IBX_HM_PURCHASE_DT_PR'].astype('str')
# df2['TU_MTG_OPEN_DT_1']               = df2['TU_MTG_OPEN_DT_1'].astype('str')


df2['AARP_IBX_BASE_RECORD_VERIFY_DT'] = df2['AARP_IBX_BASE_RECORD_VERIFY_DT'].str[:4] + ('0'+df2['AARP_IBX_BASE_RECORD_VERIFY_DT'].str[-3]) + '01'
df2['AARP_IBX_BASE_RECORD_VERIFY_DT'].values[df2['AARP_IBX_BASE_RECORD_VERIFY_DT'] == 'nan0n01'] = np.nan

# df2['TU_MTG_OPEN_DT_1'] = df2['TU_MTG_OPEN_DT_1'].str[:4] + ('0'+df2['TU_MTG_OPEN_DT_1'].str[-3]) + '01'
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == 'nan0n01'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20200001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20210001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20120001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20190001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20160001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20170001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20150001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20110001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20100001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20130001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20140001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20180001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20090001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20060001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20050001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20040001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20030001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20020001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20070001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20080001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20010001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19980001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19930001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19970001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19990001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '20000001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19960001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19950001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19920001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19940001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19880001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19910001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19830001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19800001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19870001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19890001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19860001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19820001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19850001'] = np.nan
# df2['TU_MTG_OPEN_DT_1'].values[df2['TU_MTG_OPEN_DT_1'] == '19840001'] = np.nan



df2['AARP_IBX_HM_LOAN_DT_1'] = df2['AARP_IBX_HM_LOAN_DT_1'].str[:4] + (df2['AARP_IBX_HM_LOAN_DT_1'].str[-4:-2]) + '01'
df2['AARP_IBX_HM_LOAN_DT_1'].values[df2['AARP_IBX_HM_LOAN_DT_1'] == 'nann01'] = np.nan
df2['AARP_IBX_HM_LOAN_DT_1'].values[df2['AARP_IBX_HM_LOAN_DT_1'] == '20160001'] = np.nan
df2['AARP_IBX_HM_LOAN_DT_1'].values[df2['AARP_IBX_HM_LOAN_DT_1'] == '20100001'] = np.nan
df2['AARP_IBX_HM_LOAN_DT_1'].values[df2['AARP_IBX_HM_LOAN_DT_1'] == '20060001'] = np.nan


df2['AARP_IBX_HM_PURCHASE_DT_PR'] = df2['AARP_IBX_HM_PURCHASE_DT_PR'].str[:4] + (df2['AARP_IBX_HM_PURCHASE_DT_PR'].str[-4:-2]) + '01'
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == 'nann01'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19900001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20030001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20000001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19950001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20070001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20060001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20050001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20040001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20020001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19980001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19990001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20010001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19970001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19960001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20080001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19800001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19930001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20090001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19870001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19910001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19940001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19710001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20100001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19920001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19770001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19860001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19730001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19750001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19890001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19760001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20110001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19720001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19780001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19850001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19810001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20130001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19740001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19790001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19820001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20150001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19880001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19840001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20120001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19700001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19680001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20140001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19830001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20170001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19670001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20160001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19690001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19660001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19640001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19620001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19630001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19650001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20180001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19580001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19610001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19520001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '20190001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19570001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19590001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19310001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19270001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19330001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19370001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19400001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19600001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19180001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19230001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19100001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19500001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19320001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19460001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19480001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19250001'] = np.nan
df2['AARP_IBX_HM_PURCHASE_DT_PR'].values[df2['AARP_IBX_HM_PURCHASE_DT_PR'] == '19280001'] = np.nan


# df2['AARP_DMA_CD']=df2['AARP_DMA_CD'].replace('U', '')
# df2['AARP_DMA_CD']=pd.to_numeric(df2['AARP_DMA_CD'])

df2['AARP_IBX_PASDSF_DELIVRBLTY_IDT']=df2['AARP_IBX_PASDSF_DELIVRBLTY_IDT'].replace('X', '')
df2['AARP_IBX_PASDSF_DELIVRBLTY_IDT']=pd.to_numeric(df2['AARP_IBX_PASDSF_DELIVRBLTY_IDT'])

df2['AARP_NUM_CTCT_AARP_12M_AGG_HH']=df2['AARP_NUM_CTCT_AARP_12M_AGG_HH'].replace('*', '')
df2['AARP_NUM_CTCT_AARP_12M_AGG_HH']=pd.to_numeric(df2['AARP_NUM_CTCT_AARP_12M_AGG_HH'])

df2['AARP_NUM_HISTTRPRT_DISC_AGG_HH']=df2['AARP_NUM_HISTTRPRT_DISC_AGG_HH'].replace('*', '')
df2['AARP_NUM_HISTTRPRT_DISC_AGG_HH']=pd.to_numeric(df2['AARP_NUM_HISTTRPRT_DISC_AGG_HH'])

df2['AARP_NUM_HISTTRPRT_LFSTYL_AGG_HH']=df2['AARP_NUM_HISTTRPRT_LFSTYL_AGG_HH'].replace('*', '')
df2['AARP_NUM_HISTTRPRT_LFSTYL_AGG_HH']=pd.to_numeric(df2['AARP_NUM_HISTTRPRT_LFSTYL_AGG_HH'])

In [10]:
param.loc[len(param)]=['AARP_IBX_RET_PURCH_ACT_DT_CATEGORY1', 'object', '', '', '', 'aarp']
param.loc[len(param)]=['AARP_IBX_RET_PURCH_ACT_DT_CATEGORY2', 'object', '', '', '', 'aarp']
param.loc[len(param)]=['AARP_IBX_RET_PURCH_ACT_DT_CATEGORY3', 'object', '', '', '', 'aarp']
param.loc[len(param)]=['AARP_IBX_RET_PURCH_ACT_DT_CATEGORY4', 'object', '', '', '', 'aarp']
param.loc[len(param)]=['AARP_IBX_RET_PURCH_ACT_DT_CATEGORY5', 'object', '', '', '', 'aarp']

In [ ]:
# test data logic

# testing_df=df2[['DROP_DATE', 'AARP_MEM_ORIGIN_DT']]
# testing_df['NEW_DROP_DATE']=pd.to_datetime(testing_df['DROP_DATE'], format = '%Y%m%d')
# testing_df['NEW_DROP_DATE'] = pd.to_datetime(testing_df['NEW_DROP_DATE']) - pd.offsets.MonthBegin()

# testing_df['NEW_AARP_MEM_ORIGIN_DT']=pd.to_datetime(testing_df['AARP_MEM_ORIGIN_DT'], format = '%Y-%m-%d')

# testing_df['NEW_AARP_MEM_ORIGIN_DT_d']=(( pd.to_datetime(testing_df.NEW_DROP_DATE) - pd.to_datetime(testing_df.NEW_AARP_MEM_ORIGIN_DT))/np.timedelta64(1, 'M'))
# testing_df['NEW_AARP_MEM_ORIGIN_DT_d1']=round(testing_df['NEW_AARP_MEM_ORIGIN_DT_d'])

# testing_df.head(10)


In [29]:
#date variables sourcing from param file and calculate the date diff

datedf = param[param['isdate'] == 'Y']

date_lst=[]

for i in datedf['columns']:
    date_lst.append(i)

date_lst.append('DROP_DATE')

date_data = df2[date_lst]
date_data['DROP_DATE']=pd.to_datetime(date_data['DROP_DATE'], format = '%Y%m%d')
#aligning drop date to first of every month
date_data['DROP_DATE']=pd.to_datetime(date_data['DROP_DATE']) - pd.offsets.MonthBegin()

datedict = dict(zip(datedf['columns'],datedf['dateformat']))

for i in date_data.columns:
        if i in datedict.keys():
            fmt = datedict.get(i)
            date_data[i] = pd.to_datetime(date_data[i], format = fmt)
            date_data[i] = pd.to_datetime(date_data[i])
#             date_data[i] = pd.to_datetime(date_data[i]) + pd.offsets.MonthBegin()
        
            newcol = i+'_d'
            date_data[newcol] = (pd.to_datetime(date_data.DROP_DATE) - pd.to_datetime(date_data[i]))/np.timedelta64(1, 'M')
            date_data[newcol] = round(date_data[newcol])
            
date_data1 = date_data.select_dtypes(include='number')

#create missing flag for date dataset to capture the exact cols for missing
missing_flag_dt = pd.DataFrame()
for i in date_data1.columns:
    new_col_name_dt = i+'_st'
    missing_flag_dt[new_col_name_dt] = np.where(pd.isnull(date_data1[i].values), 1, 0)
            
#num variables sourcing from param file and do the necessary steps

num_cols=param[param['datatype']=='int64']
num_cols

num_lst=[]

for i in num_cols['columns']:
    num_lst.append(i)

num_data=df2[num_lst]

vars_with_no_variance=[]
for i in num_data.columns:
    if num_data[i].min() == num_data[i].max():
        vars_with_no_variance+=[i]

num_data.drop(vars_with_no_variance,axis=1,inplace=True)
num_data.shape

num_data1=num_data.copy()
num_data1.shape

#create missing flag for date dataset to capture the exact cols for missing
missing_flag_num = pd.DataFrame()
for i in num_data1.columns:
    new_col_name_num = i+'_st'
    missing_flag_num[new_col_name_num] = np.where(pd.isnull(num_data1[i].values), 1, 0)
    
print(num_data1.shape)
print(missing_flag_num.shape)

#char variables sourcing from param file and make them


cat_cols=param[param['datatype']=='object']
cat_cols

cat_lst=[]

for i in cat_cols['columns']:
    cat_lst.append(i)

cat_data=df2[cat_lst]

#create missing flag for date dataset to capture the exact cols for missing
missing_flag_cat = pd.DataFrame()
for i in cat_data.columns:
    new_col_name_cat = i+'_st'
    missing_flag_cat[new_col_name_cat] = np.where(pd.isnull(cat_data[i].values), 1, 0)

for i in cat_data.columns:
    cat_data[i] = cat_data[i].fillna('MISS')

(852236, 553)
(852236, 553)


In [30]:
print(cat_data.shape)
print(num_data1.shape)
print(date_data1.shape)
print(missing_flag_cat.shape)
print(missing_flag_num.shape)
print(missing_flag_dt.shape)

(852236, 272)
(852236, 553)
(852236, 21)
(852236, 272)
(852236, 553)
(852236, 21)


In [31]:
aarp_ds_tu_final=pd.concat([num_data1, date_data1, cat_data, df2[['SEQ_NO', 'RESPONSE_CNT', 'RESPONSE_DATE', 'CONVERSION_CNT',
                            'CONVERSION_DATE', 'MONTH_YR', 'PSEUDO_INDIVIDUAL_ID', 'PSEUDO_HOUSEHOLD_ID', 'PSEUDO_RESIDENCE_ID',
                            'DROP_DATE', 'RANDOM_CELL_CODE', 'RANDOM_EXTRA', 'RANDOM_EXTRA_ALL_WEEKS', 'state', 'zip_code']]], axis=1)
aarp_ds_tu_final.shape

(852236, 861)

In [39]:
missing_flag_final=pd.concat([missing_flag_num, missing_flag_dt,missing_flag_cat, df2[['SEQ_NO', 'RESPONSE_CNT', 'CONVERSION_CNT']]], axis=1)
missing_flag_final.shape

(852236, 849)

In [40]:
#downgraded data with 20% response data
rdms = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/Codes/randoms_20%.csv')
print(rdms.shape)

aarp_ds_tu_final_1 = aarp_ds_tu_final[aarp_ds_tu_final['SEQ_NO'].isin(rdms['SEQ_NO'])]
print(aarp_ds_tu_final_1.shape)

#creat train for response
np.random.seed(1)
df_resp_train = aarp_ds_tu_final_1.sample(frac = 0.7)
print(df_resp_train.shape)

#create test for response
df_resp_test = aarp_ds_tu_final_1.drop(df_resp_train.index)
print(df_resp_test.shape)

(315225, 1256)
(315225, 861)
(220658, 861)
(94567, 861)


In [41]:
#exclude id, response, conversion variables from data manipulation
df_resp_train1=df_resp_train.drop(columns=['SEQ_NO', 'RESPONSE_CNT', 'RESPONSE_DATE', 'CONVERSION_CNT',
                            'CONVERSION_DATE', 'MONTH_YR', 'PSEUDO_INDIVIDUAL_ID', 'PSEUDO_HOUSEHOLD_ID', 'PSEUDO_RESIDENCE_ID',
                            'DROP_DATE', 'RANDOM_CELL_CODE', 'RANDOM_EXTRA', 'RANDOM_EXTRA_ALL_WEEKS', 'state', 'zip_code'])

In [42]:
outlier_trt_num=[]
mvi_trt_num=[]

numeric_cols=df_resp_train1.select_dtypes(include=np.number).columns.tolist()
numeric_data=df_resp_train1[numeric_cols]

for i in numeric_data.columns:
    numeric_data[i] = np.where(numeric_data[i]>np.percentile(numeric_data[i].dropna(),q=99),np.percentile(numeric_data[i].dropna(),q=99),numeric_data[i])
    numeric_data[i] = np.where(numeric_data[i]<np.percentile(numeric_data[i].dropna(),q=1),np.percentile(numeric_data[i].dropna(),q=1),numeric_data[i])
    nm= i , (numeric_data[i].quantile(0.99)), (numeric_data[i].quantile(0.01))
    outlier_trt_num.append(nm)


numeric_data1=numeric_data.copy()
numeric_data1.shape

for i in numeric_data1.columns:
    numeric_data1[i] = numeric_data1[i].fillna(numeric_data1[i].median())
    mvi_num = i , (numeric_data1[i].median())
    mvi_trt_num.append(mvi_num)

In [44]:
#calculate woe on character data
character_cols=df_resp_train1.select_dtypes(include=np.object).columns.tolist()
character_data=df_resp_train1[character_cols]
character_data1=pd.concat([character_data, df_resp_train['RESPONSE_CNT']], axis=1)

In [ ]:
# d_woe.to_csv('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/Modeldata_Train_WOE_05Jun23.csv')
# d_woe.shape

# check

In [3]:
#downgraded data with 20% response data
rdms = pd.read_csv('/mnt/data/prod/hig/projects/models/model_build_2023/Codes/randoms_20%.csv')
print(rdms.shape)

(315225, 1256)


In [4]:
rdms['RESPONSE_CNT'].value_counts(dropna=False)

0    252180
1     63045
Name: RESPONSE_CNT, dtype: int64

In [20]:
rdms['DROP_DATE'].astype(str).str[:6].value_counts(dropna=False).sort_index().reset_index()

index  DROP_DATE
0  202201      39903
1  202202      40522
2  202203      39960
3  202204      38840
4  202205      39885
5  202206      37149
6  202207      37898
7  202208      41068

In [42]:
character_data1 = rdms[['AARP_IBX_ETHNIC_ROLLUPCD_ETECH','RESPONSE_CNT']]
character_data1.shape

(315225, 2)

In [43]:
character_data1

AARP_IBX_ETHNIC_ROLLUPCD_ETECH  RESPONSE_CNT
0                                   K             1
1                                   L             1
2                                   K             1
3                                   K             1
4                                 NaN             1
...                               ...           ...
315220                              A             0
315221                              K             0
315222                              L             0
315223                              K             0
315224                              K             0

[315225 rows x 2 columns]

In [44]:

newDF,woeDF, d_woe = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
count=0
#Extract Column Names
cols = character_data1.columns
target = 'RESPONSE_CNT'
bins=10
show_woe = True
#Run WOE and IV on all the independent variables
for ivars in cols[~cols.isin([target])]:
    if (character_data1[ivars].dtype.kind in 'bifc') and (len(np.unique(character_data1[ivars]))>10):
        print('in int')
        binned_x = pd.qcut(character_data1[ivars], bins,  duplicates='drop')
        d0 = pd.DataFrame({'x': binned_x, 'y': character_data1[target]})
    else:
        print('in obj')
        d0 = pd.DataFrame({'x': character_data1[ivars], 'y': character_data1[target]})


    # Calculate the number of events in each group (bin)
    d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
    d.columns = ['Cutoff', 'N', 'Events']

    # Calculate % of events in each group.
    d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()

    # Calculate the non events in each group.
    d['Non-Events'] = d['N'] - d['Events']
    # Calculate % of non events in each group.
    d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()

    # Calculate WOE by taking natural log of division of % of non-events and % of events
    d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
    d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
    d.insert(loc=0, column='Variable', value=ivars)
    d_woe=pd.concat([d_woe, d])
    #print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
    dic = dict(zip(d['Cutoff'],d['WoE']))
    character_data1[ivars] = character_data1[ivars].map(dic)
    temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
    newDF=pd.concat([newDF,temp], axis=0)
    woeDF=pd.concat([woeDF,d], axis=0)
    count = count+1

#     #Show WOE Table
#     if show_woe == True:
#         print(d)
# return newDF
print(count)

in obj
1


In [46]:
d['Events'].sum()

59686

In [49]:
np.maximum(d['Events'],0.5)/d['Events'].sum()

0     0.133147
1     0.004122
2     0.011175
3     0.001206
4     0.056982
5     0.002346
6     0.017307
7     0.000335
8     0.004892
9     0.016604
10    0.619458
11    0.042858
12    0.000955
13    0.011091
14    0.077522
Name: Events, dtype: float64

In [26]:
d_woe

Variable Cutoff       N  Events  % of Events  \
0   AARP_IBX_ETHNIC_ROLLUPCD_ETECH      A   26539    7947     0.133147   
1   AARP_IBX_ETHNIC_ROLLUPCD_ETECH      B    1083     246     0.004122   
2   AARP_IBX_ETHNIC_ROLLUPCD_ETECH      C    2348     667     0.011175   
3   AARP_IBX_ETHNIC_ROLLUPCD_ETECH      D     350      72     0.001206   
4   AARP_IBX_ETHNIC_ROLLUPCD_ETECH      E   15512    3401     0.056982   
5   AARP_IBX_ETHNIC_ROLLUPCD_ETECH      F     688     140     0.002346   
6   AARP_IBX_ETHNIC_ROLLUPCD_ETECH      G    6933    1033     0.017307   
7   AARP_IBX_ETHNIC_ROLLUPCD_ETECH      H      69      20     0.000335   
8   AARP_IBX_ETHNIC_ROLLUPCD_ETECH      I    1049     292     0.004892   
9   AARP_IBX_ETHNIC_ROLLUPCD_ETECH      J    4495     991     0.016604   
10  AARP_IBX_ETHNIC_ROLLUPCD_ETECH      K  207936   36973     0.619458   
11  AARP_IBX_ETHNIC_ROLLUPCD_ETECH      L   13840    2558     0.042858   
12  AARP_IBX_ETHNIC_ROLLUPCD_ETECH      M     120      57     0.000955   
13  AARP_IBX_ETHNIC_ROLLUPCD_ETECH      N    3910     662     0.011091   
14  AARP_IBX_ETHNIC_ROLLUPCD_ETECH      O   18227    4627     0.077522   

    Non-Events  % of Non-Events       WoE        IV  
0        18592         0.076380  0.555725  0.031546  
1          837         0.003439  0.181170  0.000124  
2         1681         0.006906  0.481308  0.002055  
3          278         0.001142  0.054707  0.000004  
4        12111         0.049755  0.135617  0.000980  
5          548         0.002251  0.041029  0.000004  
6         5900         0.024239 -0.336823  0.002335  
7           49         0.000201  0.509574  0.000068  
8          757         0.003110  0.453053  0.000807  
9         3504         0.014395  0.142716  0.000315  
10      170963         0.702358 -0.125597  0.010412  
11       11282         0.046349 -0.078321  0.000273  
12          63         0.000259  1.305579  0.000909  
13        3248         0.013344 -0.184867  0.000416  
14       13600         0.055872  0.327501  0.007090

In [47]:
df_resp_cons_train=pd.concat([numeric_data1, character_data1, df_resp_train[['SEQ_NO', 'RESPONSE_CNT', 'RESPONSE_DATE', 'CONVERSION_CNT',
                            'CONVERSION_DATE', 'MONTH_YR', 'PSEUDO_INDIVIDUAL_ID', 'PSEUDO_HOUSEHOLD_ID', 'PSEUDO_RESIDENCE_ID',
                            'DROP_DATE', 'RANDOM_CELL_CODE', 'RANDOM_EXTRA', 'RANDOM_EXTRA_ALL_WEEKS', 'state', 'zip_code']]], axis=1)

In [49]:
# These modification are executed to avoid data issue while mapping woe

df_resp_test['AARP_MEM_TYPE'].values[df_resp_test['AARP_MEM_TYPE'] == 1.0] = '1.0'

df_resp_test['AARP_MEM_STATUS'].values[df_resp_test['AARP_MEM_STATUS'] == 0] = '0'
df_resp_test['AARP_MEM_STATUS'].values[df_resp_test['AARP_MEM_STATUS'] == '0.0'] = '0'
df_resp_test['AARP_MEM_STATUS'].values[df_resp_test['AARP_MEM_STATUS'] == 5] = '5'
df_resp_test['AARP_MEM_STATUS'].values[df_resp_test['AARP_MEM_STATUS'] == '5.0'] = '5'
df_resp_test['AARP_MEM_STATUS'].values[df_resp_test['AARP_MEM_STATUS'] == 4] = '4'
df_resp_test['AARP_MEM_STATUS'].values[df_resp_test['AARP_MEM_STATUS'] == '4.0'] = '4'

df_resp_test['AARP_NMAS_28_MEDIA'].values[df_resp_test['AARP_NMAS_28_MEDIA'] == 0.0] = '0.0'

In [52]:
aarp_ds_tu_final.to_csv('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/NewResponseData/ModelData_AARP_DS_TU_01Jun23.csv',index=False)
print(aarp_ds_tu_final.shape)

missing_flag_final.to_csv('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/NewResponseData/ModelData_MissingFlag_01Jun23.csv', index=False)
print(missing_flag_final.shape)

df_resp_cons_train = df_resp_cons_train.loc[:,~df_resp_cons_train.columns.duplicated()].copy()
df_resp_cons_train.to_csv('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/NewResponseData/ModelData_Response_Train_01Jun23.csv', index=False)
print(df_resp_cons_train.shape)

df_resp_test.to_csv('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/NewResponseData/ModelData_Response_Test_01Jun23.csv', index=False)
print(df_resp_test.shape)

df_resp_train.to_csv('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/NewResponseData/ModelData_Response_Train_RawData_01Jun23.csv', index=False)
print(df_resp_train.shape)

aarp_ds_tu_final_1.to_csv('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/NewResponseData/ModelData_Downsampledata_Conversion_01Jun23.csv', index=False)
print(aarp_ds_tu_final_1.shape)

(852236, 861)
(852236, 849)
(220658, 861)
(94567, 861)
(220658, 861)
(315225, 861)


In [53]:
d_woe.to_csv('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/NewResponseData/ModelData_Train_WOE_01Jun23.csv', index=False)

imv_median=pd.DataFrame(mvi_trt_num)
imv_median.rename(columns={0:'var',
                          1:'mvi'},inplace=True)
imv_median.to_excel('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/NewResponseData/ModelData_Train_IMV_01Jun23.xlsx', sheet_name='MVI', index=False)

outlier=pd.DataFrame(outlier_trt_num)
outlier.rename(columns={0:'var',
                          1:'max',
                       2:'min'},inplace=True)

outlier.to_excel('/mnt/data/prod/hig/projects/models/model_build_2023/ModelCode/ResponseModelMay23/NewResponseData/ModelData_Train_Outlier_01Jun23.xlsx',  sheet_name='Outlier', index=False)